# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../output_data/cities.csv")
city_data.head(10)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
8,8,tiksi,95,RU,1558378756,88,71.64,128.87,14.10,15.70
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Lat", "Lng"]].astype(float)

# Fill NaN values and convert to float
humidity = city_data["Humidity"].astype(float)

In [6]:
# Customize the size of the figure
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
plt.savefig("../output_data/map1.png", dpi=600)
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#my prefered weather criteria: temp 50-65, humity less than 50, wind speed under 15
hotel_df = city_data.loc[(city_data['Max Temp'] <65) & (city_data['Humidity'] <30) & (city_data['Wind Speed'] <15)]
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
42,42,yellowknife,75,CA,1558378762,26,62.45,-114.38,55.40,8.05
156,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99
362,362,datong,0,CN,1558378858,23,40.09,113.30,55.32,9.44
387,387,el alto,75,BO,1558378857,19,-16.50,-68.19,57.20,14.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_name = hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\brand\AppData\Local\Temp/ipykernel_40592/3001886289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_name = hotel_df["Hotel Name"] = ""


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
42,42,yellowknife,75,CA,1558378762,26,62.45,-114.38,55.40,8.05,
156,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17,
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50,
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99,
362,362,datong,0,CN,1558378858,23,40.09,113.30,55.32,9.44,
387,387,el alto,75,BO,1558378857,19,-16.50,-68.19,57.20,14.99,


In [9]:
#loop over to grab geocoordinates
hotel = []
for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    coordinates = f"{lat},{lng}"
    target_search = "lodging"
    target_radius = 5000
    
# set up a parameters dictionary
    params = {"location": coordinates,
              "keyword": target_search, 
              "radius": target_radius,  
              "key": g_key
             }

# # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# # run a request using our params dictionary and append hotel name into empty list
    response = requests.get(base_url, params=params)
    hotel_response = response.json()
   
    try:
        print(hotel_response["results"][0]["name"])
        hotel.append(hotel_response["results"][0]["name"])
    except:
        print("Cant' locate hotel.")
    

Blachford Lake Lodge Office
Igdlo Guesthouse
玉门宾馆
Jiuquan Hotel
Best Western Golden Prairie Inn & Suites
Yunzhong Yi Traditional Inn
Paitití Hostel


In [10]:
hotel_df["Hotel Name"] = hotel
hotel_df

C:\Users\brand\AppData\Local\Temp/ipykernel_40592/3005990346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
42,42,yellowknife,75,CA,1558378762,26,62.45,-114.38,55.40,8.05,Blachford Lake Lodge Office
156,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17,Igdlo Guesthouse
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,玉门宾馆
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50,Jiuquan Hotel
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99,Best Western Golden Prairie Inn & Suites
362,362,datong,0,CN,1558378858,23,40.09,113.30,55.32,9.44,Yunzhong Yi Traditional Inn
387,387,el alto,75,BO,1558378857,19,-16.50,-68.19,57.20,14.99,Paitití Hostel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

from ipywidgets.embed import embed_minimal_html

# Display figure
fig.add_layer(markers)
#plt.savefig("../output_data/map2.png", dpi=600)
fig
embed_minimal_html('map2.html', views=[fig])